<a href="https://colab.research.google.com/github/AIWintermuteAI/Bertelsmann-Tech-Scholarship-Challenge-Course---AI-Track-Nanodegree-Program/blob/master/CatDogMNIST_(Exercise).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatDogMNIST

We'll be using a [dataset of cat and dog photos](https://www.kaggle.com/c/dogs-vs-cats) available from Kaggle. Here are a couple example images:

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/intro-to-pytorch/assets/dog_cat.png?raw=1'>

It's a fun exercise to use the knowledge about building FC neural netowrks with pytorch and transform cats-vs-dogs Kaggle dataset into a MNIST like dataset that this model can (sort of) learn.

We'll start by downloading and unzipping the dataset.

In [0]:
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip -q Cat_Dog_data.zip 

Next we'll do the necesary imports. Nothing special, mostly the stuff we had in the original notebook by Udacity, plus Python random module.

In [0]:
import matplotlib.pyplot as plt
import random
import torch
from torchvision import datasets, transforms

Create the training and testing loaders --- use the same transform for both, since using image augumentations is too hardcore for our simple fully-connected model.

In [0]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data. The transform is the same for both, it includes converting to grayscale, resizing, center-crop and finally conversion to tensor.

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=transform)
test_data = datasets.ImageFolder(data_dir + '/test', transform=transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32,shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32,shuffle=True)

And check the results in the first batch. Well, it's not too bad. I can distinguish between cats and dogs in most images.

In [0]:
# change this to the trainloader or testloader 
data_iter = iter(trainloader)
images, labels = next(data_iter)

#TODO use matplotlib.imshow to display 4 random images from the first batch



Great! Let's define our model. We'll use the model similiar to the one we used before for MNIST and FashionMNIST. The only significant change is adding a multiplier to optionally make the network wider.

In [0]:
from torch import nn, optim
import torch.nn.functional as F

multiplier = 1
dropout = 0.2

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        #TODO defein the FC layers with multiplier
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 30
steps = 0

And finally let our little brave model start the training. I had the following results for 0.5, 1 and 2 multiplier after 30 epochs.

In [0]:
train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
      test_loss = 0
      accuracy = 0
      with torch.no_grad():
        model.eval()
        for images, labels in testloader:
          ps = model(images)
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == labels.view(*top_class.shape)
          test_loss += criterion(ps, labels)
          accuracy += torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
        print("Epoch: {}/{}.. ".format(e+1, epochs),
        "Training Loss: {:.3f}.. ".format(train_losses[-1]),
        "Test Loss: {:.3f}.. ".format(test_losses[-1]),
        "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))